## Notebook for first stage YOLO model training of 2-stage model pipline

In [ ]:
import cv2
from pathlib import Path

# Target size
TARGET_SIZE = 720
ORIGINAL_SIZE = 1440
SCALE = TARGET_SIZE / ORIGINAL_SIZE

# Folders to process
splits = ["train", "val", "test"]
base_dir = Path("dataset")

def process_image_and_label(img_path: Path, lbl_path: Path):
    # Resize image
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[WARN] Failed to read: {img_path}")
        return
    resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_AREA)
    cv2.imwrite(str(img_path), resized)

    # Adjust label
    if lbl_path.exists():
        with open(lbl_path, "r") as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, x, y, w, h = map(float, parts)

            # Since all coordinates are normalized, they don't change.
            # But if they were based on pixel sizes, we’d scale:
            # (not needed here since images are same aspect & normalized)

            new_lines.append(f"{int(cls)} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        with open(lbl_path, "w") as f:
            f.write("\n".join(new_lines) + "\n")
    else:
        print(f"[WARN] Missing label for {img_path.name}")

# Process each split
for split in splits:
    image_dir = base_dir / "images" / split
    label_dir = base_dir / "labels" / split
    for img_path in image_dir.glob("*.jpg"):
        lbl_path = label_dir / (img_path.stem + ".txt")
        process_image_and_label(img_path, lbl_path)


In [ ]:
# Check cuda support
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

In [1]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="D:/CSE493G-Project/TwoStageYOLOTrain/dataset/dataset.yaml",  # 必须是完整路径
    imgsz=720,
    epochs=100,
    batch=64,
    device=0,
)

Ultralytics 8.3.144  Python-3.12.7 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/CSE493G-Project/TwoStageYOLOTrain/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=720, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

train: Scanning D:\CSE493G-Project\TwoStageYOLOTrain\dataset\labels\train.cache... 1110 images, 10 backgrounds, 0 corrupt: 100%|██████████| 1110/1110 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 277.667.4 MB/s, size: 132.6 KB)


val: Scanning D:\CSE493G-Project\TwoStageYOLOTrain\dataset\labels\val.cache... 130 images, 20 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<?, ?it/s]

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Yuzhi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Yuzhi\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in 

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



...

                   all        130        110      0.869      0.855      0.874      0.669



100 epochs completed in 0.289 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 6.3MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics 8.3.144  Python-3.12.7 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.92it/s]


                   all        130        110      0.888      0.882      0.896      0.694
Speed: 0.3ms preprocess, 2.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000274C3CD2D80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480